# FAO exercice

In [112]:
# init
import pandas as pd

df_anim = pd.read_csv("FAOSTAT_animal.csv")
df_veg = pd.read_csv("FAOSTAT_vegetal.csv")
df_pop = pd.read_csv("FAOSTAT_population.csv")

## Explore the initial data

In [113]:
print(f"\nThe first lines of df_anim:\n", df_anim.head())
print(f"\nThe first lines of df_veg:\n", df_veg.head())
print(f"\nThe first lines of df_pop:\n", df_pop.head())


The first lines of df_anim:
   Domain Code                 Domain  Area Code         Area  Element Code  \
0         FBS  Food Balances (2010-)          2  Afghanistan           645   
1         FBS  Food Balances (2010-)          2  Afghanistan           645   
2         FBS  Food Balances (2010-)          2  Afghanistan           664   
3         FBS  Food Balances (2010-)          2  Afghanistan           664   
4         FBS  Food Balances (2010-)          2  Afghanistan           674   

                                  Element  Item Code         Item  Year Code  \
0     Food supply quantity (kg/capita/yr)       2731  Bovine Meat       2018   
1     Food supply quantity (kg/capita/yr)       2731  Bovine Meat       2019   
2           Food supply (kcal/capita/day)       2731  Bovine Meat       2018   
3           Food supply (kcal/capita/day)       2731  Bovine Meat       2019   
4  Protein supply quantity (g/capita/day)       2731  Bovine Meat       2018   

   Year             

In [114]:
print(f"This is the shape of df_anim : {df_anim.shape}")
print(f"This is the shape of df_veg : {df_veg.shape}")
print(f"This is the shape of df_pop : {df_pop.shape}")

This is the shape of df_anim : (30725, 12)
This is the shape of df_veg : (90576, 12)
This is the shape of df_pop : (356, 12)


## Add a column type to *df_veg* and *df_anim*

In [115]:
df_anim = df_anim.assign(type="animal_product")
df_veg = df_veg.assign(type="vegetal_product")

## Combine *df_anim* and *df_veg*

In [116]:
product = pd.concat([df_anim, df_veg], ignore_index=True)

## Only keep country code, country, year and population in df_pop

In [117]:
df_pop = df_pop.loc[:, ["Area Code", "Area", "Year", "Value"]]

## Rename columns

In [118]:
def change_name(name):
  return name.lower().replace(" ", "_")

df_pop = df_pop.rename(columns=change_name)
product = product.rename(columns=change_name)

## Merge *df_pop* and *product*

In [119]:
print(product.shape)
df = pd.merge(product, df_pop, on=["area", "year"], suffixes=("_product", "_pop"), how="inner")
print(df.shape)

(121301, 13)
(121301, 15)


In [120]:
df.head()

,domain_code,domain,area_code_product,area,element_code,element,item_code,item,year_code,year,unit,value_product,type,area_code_pop,value_pop
0,FBS,Food Balances (2010-),2,Afghanistan,645,Food supply quantity (kg/capita/yr),2731,Bovine Meat,2018,2018,kg,2.62,animal_product,2,37172
1,FBS,Food Balances (2010-),2,Afghanistan,664,Food supply (kcal/capita/day),2731,Bovine Meat,2018,2018,kcal/capita/day,15.00,animal_product,2,37172
2,FBS,Food Balances (2010-),2,Afghanistan,674,Protein supply quantity (g/capita/day),2731,Bovine Meat,2018,2018,g/capita/day,1.09,animal_product,2,37172
3,FBS,Food Balances (2010-),2,Afghanistan,684,Fat supply quantity (g/capita/day),2731,Bovine Meat,2018,2018,g/capita/day,1.20,animal_product,2,37172
4,FBS,Food Balances (2010-),2,Afghanistan,645,Food supply quantity (kg/capita/yr),2732,Mutton & Goat Meat,2018,2018,kg,4.26,animal_product,2,37172


## Check DataTypes and modify if needed

In [121]:
df.dtypes

domain_code           object
domain                object
area_code_product      int64
area                  object
element_code           int64
element               object
item_code              int64
item                  object
year_code              int64
year                   int64
unit                  object
value_product        float64
type                  object
area_code_pop          int64
value_pop              int64
dtype: object

## Check missing values and visualize them

In [122]:
null_values = df.isnull().sum(axis=1)

In [123]:
import missingno as msno

In [ ]:
msno.bar(df)

## Find if there is weird values and drop them

In [125]:
weird_values = df.loc[(df.value_product < 0) | (df.value_pop < 0)]

In [126]:
print(df.shape)
df.drop(weird_values.index, inplace=True)
print(df.shape)

(121301, 15)
(121299, 15)


## Improve column's names and clean

In [127]:
df = df.rename(columns={"area_code_product": "area_code", "value_pop": "population"})
df = df.drop(["area_code_pop", "domain", "domain_code", "year_code"], axis=1)
df.head()

,area_code,area,element_code,element,item_code,item,year,unit,value_product,type,population
0,2,Afghanistan,645,Food supply quantity (kg/capita/yr),2731,Bovine Meat,2018,kg,2.62,animal_product,37172
1,2,Afghanistan,664,Food supply (kcal/capita/day),2731,Bovine Meat,2018,kcal/capita/day,15.00,animal_product,37172
2,2,Afghanistan,674,Protein supply quantity (g/capita/day),2731,Bovine Meat,2018,g/capita/day,1.09,animal_product,37172
3,2,Afghanistan,684,Fat supply quantity (g/capita/day),2731,Bovine Meat,2018,g/capita/day,1.20,animal_product,37172
4,2,Afghanistan,645,Food supply quantity (kg/capita/yr),2732,Mutton & Goat Meat,2018,kg,4.26,animal_product,37172


## Show Area's unique values

In [128]:
df.area.unique()

array(['Afghanistan', 'Albania', 'Algeria', 'Angola',
       'Antigua and Barbuda', 'Argentina', 'Armenia', 'Australia',
       'Austria', 'Azerbaijan', 'Bahamas', 'Bangladesh', 'Barbados',
       'Belarus', 'Belgium', 'Belize', 'Benin',
       'Bolivia (Plurinational State of)', 'Bosnia and Herzegovina',
       'Botswana', 'Brazil', 'Bulgaria', 'Burkina Faso', 'Burundi',
       'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada',
       'Central African Republic', 'Chad', 'Chile',
       'China, Hong Kong SAR', 'China, Macao SAR', 'China, mainland',
       'China, Taiwan Province of', 'Colombia', 'Comoros', 'Congo',
       'Costa Rica', "C?te d'Ivoire", 'Croatia', 'Cuba', 'Cyprus',
       'Czechia', "Democratic People's Republic of Korea",
       'Democratic Republic of the Congo', 'Denmark', 'Djibouti',
       'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt',
       'El Salvador', 'Estonia', 'Eswatini', 'Ethiopia', 'Fiji',
       'Finland', 'France', 'French Polynesia', 'Gabon', 'Gamb

## Rename *area_code* and *area* to clearer names

In [129]:
df = df.rename(columns={"area_code": "area_country", "area": "country"})

In [136]:
df = df.pivot(index=["area_country", "country", "year", "population", "item", "type"],columns="element", values="value_product")
df = df.reset_index()
df.head()

element,area_country,country,year,population,item,type,Fat supply quantity (g/capita/day),Food supply (kcal/capita/day),Food supply quantity (kg/capita/yr),Protein supply quantity (g/capita/day)
0,1,Armenia,2018,2952,Apples and products,vegetal_product,0.12,20.0,14.49,0.05
1,1,Armenia,2018,2952,"Aquatic Animals, Others",animal_product,0.00,0.0,0.00,0.00
2,1,Armenia,2018,2952,Aquatic Plants,animal_product,0.00,0.0,0.00,0.00
3,1,Armenia,2018,2952,Bananas,vegetal_product,0.06,13.0,7.61,0.15
4,1,Armenia,2018,2952,Barley and products,vegetal_product,0.03,6.0,0.72,0.20


## Modify values so that 